# Example: Model selection

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor

from dskit.model_selection import parameter_grid, compare_models

%matplotlib inline

Specifying parameter grid 

In [ ]:
grid_specs = (
    (ElasticNet, {'alpha': [0.01, 0.1, 1, 10], 'l1_ratio': [0.3, 0.5, 0.7, 0.9]}),
    (RandomForestRegressor, {'max_depth': [5, 10, 50, 100, 200, 500]})
)

Formatting grid specifications 

In [ ]:
models_and_parameters = parameter_grid(grid_specs)

Create an esitmator pipeline

In [ ]:
pipel = make_pipeline(
    PCA(n_components=0.99, random_state=0),
    estimator(random_state=0)
)

Compare models and report the best alternative

In [ ]:
results = compare_models(
    X, y, pipe, test_size=0.3, folds=10, scoring='neg_mean_squared_error'
)
report_best_model(results)

In [117]:
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array, check_X_y

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline


from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [2]:
X, y = make_regression()

In [ ]:
"""X (array-like): An (n x m) array of feature samples.
y (array-like): An (n x 1) array of target samples.
test_size (float): The fraction of data used in validation.
random_state (int): The random number generator intiator.
estimator (class): The learning algorithm.
param_grid (dict): The hyperparameter grid with parameter name and iterable
    parameter values as key-value pairs.
scoring ():
folds (int): The number of folds to generate in cross validation."""

In [3]:
class Standardizer:
    """Standardize feature data by subtracting mean and dividing by
    standard deviation."""

    # NOTE: Fit standard scaler to training data to learn training data
    # parameters. Transform both training data and test data with training
    # data parameters. Thus, standardized training and test data are comparable.

    def __init__(self, scaler=StandardScaler, **kwargs):

        self._scaler = scaler(**kwargs)

    def fit(self, X, y=None, **kwargs):

        #self._scaler()
        self._scaler.fit(X)

        return self

    def transform(self, X):

        return self._scaler.transform(X)
    
    def fit_transform(self, X, y=None, **kwargs):
        
        self.fit(X, y=y, **kwargs)
        
        return self.transform(X)

In [7]:
def train_test_scaling(X, y, test_size, random_state):

    # NOTE: Should be function not class since dependent of random numer
    # intiator and thus cannot be included in pipeline.

    # Generate training and test sets.
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    # Standardize feature data.
    scaler = Standardizer()
    X_train_std = scaler.fit_transform(X_train)
    X_test_std = scaler.transform(X_test)

    return X_train_std, X_test_std, y_train, y_test

In [142]:
def nested_cross_val(*args):

    # Construct training and test splits including scaling of feature data.
    X_train_std, X_test_std, y_train, y_test = train_test_scaling(
        args[0], args[1], test_size=args[2], random_state=args[3]
    )
    # Perform cross-validated hyperparameter search.
    cv_grid = GridSearchCV(
        estimator=args[4], param_grid=args[5], scoring=args[6], cv=args[7]
    )
    cv_grid.fit(X_train_std, y_train)
    # Array of scores of the estimator for each run of the cross validation.
    cv_train_score = cross_val_score(cv_grid, X_train_std, y_train)
    cv_test_score = cross_val_score(cv_grid, X_test_std, y_test)

    return cv_train_score, cv_test_score

def model_performance_report(name, train_scores, test_scores):
    
    print('Model performance report', '\n{}'.format('-' * 25))
    print('Name: {}\nTraining scores: {} +/- {}\nTest scores: {} +/- {}'
          ''.format(name,
                    np.round(np.mean(train_scores), decimals=3),
                    np.round(np.std(train_scores), decimals=3),
                    np.round(np.mean(test_scores), decimals=3),
                    np.round(np.std(test_scores), decimals=3)))

    print('Train-test difference: {}\n'
          ''.format(np.mean(train_scores) - np.mean(test_scores))
    )

    
def parameter_grid(grid_specs, pipeline=True):
    
    models_and_parameters = {}
    for model, params in grid_specs:
        
        if pipeline:
            model_name = str(model.steps[-1][0])
        else:
            model_name = str(model.__name__).lower()
        
        models_and_parameters[model_name] = (model, {})
        for key, value in params.items():

            if pipeline:        
                param_name = ('__').join((model_name, str(key)))
            else:
                param_name = str(key)

            models_and_parameters[model_name][1][param_name] = value

    return models_and_parameters

In [146]:
grid_specs = (
    (ElasticNet, {'alpha': [0.01, 0.1, 1], 'l1_ratio': [0.3, 0.5, 0.7]}),
    (RandomForestRegressor, {'max_depth': [200, 500]})
)

elnet_pipe = make_pipeline(
    PCA(n_components=0.99, random_state=0),
    ElasticNet(random_state=0)
)

rf_pipe = make_pipeline(
    PCA(n_components=0.99, random_state=0),
    RandomForestRegressor(random_state=0)
)

pipe_grid_specs = (
    (elnet_pipe, {'alpha': [0.01, 0.1, 1], 'l1_ratio': [0.3, 0.5, 0.7]}),
    (rf_pipe, {'max_depth': [200, 500]})
)

pipes_and_parameters = parameter_grid(pipe_grid_specs, pipeline_compat=True)
models_and_parameters = parameter_grid(grid_specs, pipeline_compat=False)

In [147]:
def compare_estimators(X, y, models_and_params, scoring, test_size=0.3, folds=10):
    
    results = {}
    for model_name, (model, param_grid) in models_and_params.items():
        
        results[model_name] = {'train_scores': [], 'test_scores': []}
        # Address model stochasticity by eval across multiple random states.
        for random_state in range(2):
            
            if isinstance(model, Pipeline):
                estimator = model
            else:
                estimator = model(random_state=random_state)
            # Collect training and test scores from nested cross-validation.
            train_scores, test_scores = nested_cross_val(
                X, y, test_size, random_state, estimator, param_grid, scoring, folds
            )
            results[model_name]['train_scores'].append(np.mean(train_scores))
            results[model_name]['test_scores'].append(np.mean(test_scores))

        # Print model training and test performance.
        model_performance_report(model_name, train_scores, test_scores)

    return results

In [148]:
results = compare_estimators(
    X, y, models_and_parameters, scoring='neg_mean_squared_error'
)

/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: Conv

/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: Conv

/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: Conv

/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: Conv

/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: Conv

/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: Conv

/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: Conv

/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: Conv

/Users/severinlangberg/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Model performance report 
-------------------------
Name: elasticnet
Training scores: -13277.287 +/- 2818.367
Test scores: -16233.117 +/- 10582.638
Train-test difference: 2955.829755071085

Model performance report 
-------------------------
Name: randomforestregressor
Training scores: -17211.273 +/- 8220.855
Test scores: -20911.875 +/- 14019.213
Train-test difference: 3700.6021008152857



In [150]:
results2 = compare_estimators(
    X, y, pipes_and_parameters, scoring='neg_mean_squared_error'
)

Model performance report 
-------------------------
Name: elasticnet
Training scores: -15551.735 +/- 3053.959
Test scores: -16598.967 +/- 10853.259
Train-test difference: 1047.2326036906943

Model performance report 
-------------------------
Name: randomforestregressor
Training scores: -17406.534 +/- 6996.248
Test scores: -21283.767 +/- 13735.53
Train-test difference: 3877.233268629858



In [253]:
def report_best_model(results, criteria='variance'):
    """Determines the optimal model by evaluating the model performance results 
    according to a specified criteria.
    
    Args:
        results ():
        criteria (str, {bias, variance}): Decision rule for model performance comparison.
            Is `variance` criteria: Selects the model corresponding to the minimum difference 
            between the training and test scores. If `bias` criteria: Selects the model 
            corresponding to the maximum test score.

    """
    
    if criteria == 'bias':
        init_score = -np.float('inf')
    elif criteria == 'variance':
        init_score = np.float('inf')
    else:
        raise ValueError('Invalid evaluation criteria: `{}`'.format(criteria))
    
    best_model, best_score = None, init_score
    for model_name, scores in results.items():
        
        keys, (train, test) = list(scores.keys()), list(scores.values())
        
        if criteria == 'bias':
            score = np.max(test)
            if score > best_score:
                best_model, best_score = model_name, score
            else:
                continue
                
        elif criteria == 'variance':
            score = np.sum(np.squeeze(train) - np.squeeze(test))
            if score < best_score:
                best_model, best_score = model_name, score
            else:
                continue
    
    print('Best model report', '\n{}'.format('-' * 20))
    print('Name: {}\nCriteria: {}\nBest scores: {}'.format(best_model, 
                                                           criteria, 
                                                           best_score))
    
    return None

In [254]:
report_best_model(results, criteria='variance')

Best model report 
--------------------
Name: randomforestregressor
Criteria: variance
Best scores: 16929.752335039782


In [255]:
report_best_model(results2, criteria='variance')

Best model report 
--------------------
Name: randomforestregressor
Criteria: variance
Best scores: 13152.452250023243


In [256]:
report_best_model(results, criteria='bias')

Best model report 
--------------------
Name: elasticnet
Criteria: bias
Best scores: -16233.116911202706


In [257]:
report_best_model(results2, criteria='bias')

Best model report 
--------------------
Name: elasticnet
Criteria: bias
Best scores: -16598.96724685651
